In [1]:
from backend.retrieval.ciena_retreival import CienaRetrieval
from backend.embedder.baseEmbedder import baseEmbedder
embedding_function = baseEmbedder().embedding_function
retriael_kwargs = {
    "threshold": "0.8",
    "k": 8,
    "min_samples": 2,
    "embedder": embedding_function,
    "hybrid": True
}

ciena_retreival = CienaRetrieval(**retriael_kwargs)

/anaconda/envs/bc/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.12) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/anaconda/envs/bc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from backend.retrieval.utils import *
from langchain.document_loaders import JSONLoader
dir = './output/'
loaded_data = []
for r, d, f in os.walk(dir):
    for file in f:
        if '.json' in file and file != 'structuredData.json':
            dir_ = file.rsplit('.')[0] + '.pdf'
            file_name = os.path.join(dir, dir_, file)
            print(file_name)
            try:
                loader = JSONLoader(
                    file_path=file_name,
                    jq_schema='.[].content[]',
                    content_key="text", 
                    metadata_func=metadata_func)

                loaded_data.extend(loader.load())
                print(f"Successfully loaded file {file_name}")
            except:
                print(f"error in loading  file {file_name}")
                pass

len(loaded_data)

./output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json
Successfully loaded file ./output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json
./output/11Aug-BP_Engineering_guide.pdf/11Aug-BP_Engineering_guide.json
Successfully loaded file ./output/11Aug-BP_Engineering_guide.pdf/11Aug-BP_Engineering_guide.json
./output/16Aug-BP_Engineering_Guide.pdf/16Aug-BP_Engineering_Guide.json
Successfully loaded file ./output/16Aug-BP_Engineering_Guide.pdf/16Aug-BP_Engineering_Guide.json
./output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json
error in loading  file ./output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json
./output/24-Jan-450-3704-300-2110-UAA_21.pdf/24-Jan-450-3704-300-2110-UAA_21.json
error in loading  file ./output/24-Jan-450-3704-300-2110-UAA_21.pdf/24-Jan-450-3704-300-2110-UAA_21.json
./output/25Aug_Blue_Planet_Engineering_Guide_2208.pdf/25Aug_Blue_Planet_Engineering_Guide_2208.json
Successfully loaded file ./output/25Aug_Blue_Planet_Engineering_Guide_2208.pdf/25Aug_Blue_P

39941

In [8]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [9]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template='You are a helpful assistant that translates {input_language} to {output_language}.'))

In [3]:
loaded_data = filter_empty(loaded_data)

In [4]:
unique_docs = filter_redundant(loaded_data)

In [5]:
query = "give me a table for ciena's BPO Runtime License"
res = ciena_retreival.get_res(query= query, docs= unique_docs)

In [6]:
res = remove_duplicates(res)

In [7]:
from backend.retrieval.rereanker import Reranker
reranker = Reranker()
reranked_res = reranker.rerank(query, res)

In [11]:
reranked_res

[Document(page_content='Table 3. BPO Runtime Licenses ', metadata={'header': 'BPO runtime licenses ', 'page_number': 7, 'seq_num': 322, 'source': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ahmed1/code/Clients/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json', 'table_path': '', 'type': 'P[27]'})]

In [9]:
context = ciena_retreival.get_context(reranked_res)

In [10]:
context = list(set(context))
context

['Table 3. BPO Runtime Licenses ',
 'The following tables list the licenses to run the software application. ']

In [1]:
test = {'header': '', 'page_number': 7, 'seq_num': 321, 'source': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ahmed1/code/Clients/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json', 'table_path': 'tables/fileoutpart3.xlsx', 'type': 'Table'}

In [2]:
from backend.retrieval.ciena_retreival import CienaRetrieval
from backend.embedder.baseEmbedder import baseEmbedder
embedding_function = baseEmbedder().embedding_function
retriael_kwargs = {
    "threshold": "0.8",
    "k": 20,
    "embedder": embedding_function,
    "hybrid": True
}
ciena_retreival = CienaRetrieval(**retriael_kwargs)

/anaconda/envs/bc/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.11) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/anaconda/envs/bc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
context = []
from backend.retrieval.utils import *
table_dir = './output/10Aug-BP_Engineering_guide.pdf'
res = [test]
if 'Table'in test["type"] and test['table_path'] != '':
    context.append(md_table(os.path.join(table_dir, test['table_path'])))


print(context)

['|    | PRODUCT _x000D_   | LICENSE(S) TO ORDER _x000D_   |\n|---:|:------------------|:------------------------------|\n|  0 | BPI _x000D_       | S19-BP-FED-S1P0 _x000D_       |\n|  1 | MDSO _x000D_      | S19-LIC-BP-MDSO _x000D_       |\n|  2 | NFVO _x000D_      | S19-LIC-BP-NFVO _x000D_       |\n|  3 | ROA _x000D_       | S19-LIC-BP-ROA _x000D_        |']


In [ ]:
get_context(relevant_docs)